# Setup

In [2]:
import os
import zipfile
from pathlib import Path
from dotenv import load_dotenv


def check_env() -> str:
    if os.environ.get('KAGGLE_KERNEL_RUN_TYPE'):
        print("Running on Kaggle")
        return "kaggle"
    else:
        print("Running locally")
        return "local"


ENV = check_env()

if ENV == "kaggle":
    data_dir = Path("/kaggle/input/ka-ocr")
else:
    load_dotenv()

    from huggingface_hub import hf_hub_download

    data_dir = Path("./data")
    data_dir.mkdir(parents=True, exist_ok=True)

    hf_repo = os.getenv("HF_DATASET_REPO")
    hf_token = os.getenv("HF_TOKEN")

    if not hf_repo:
        raise ValueError("HF_DATASET_REPO not set in .env")

    # Download with automatic caching - skips if local matches remote (etag-based)
    zip_path = hf_hub_download(
        repo_id=hf_repo,
        filename="ka-ocr.zip",
        repo_type="dataset",
        token=hf_token,
        local_dir=data_dir,
    )

    # Extract only if not already extracted OR if zip is newer than extraction
    extract_marker = data_dir / ".extracted"
    zip_file = Path(zip_path)
    needs_extract = (
        not extract_marker.exists() or
        zip_file.stat().st_mtime > extract_marker.stat().st_mtime
    )

    if needs_extract:
        print("Extracting dataset...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(data_dir)
        extract_marker.touch()
        print("Extraction complete")
    else:
        print("Dataset already extracted, skipping")

print(f"\nDataset contents in {data_dir}:")
for item in data_dir.iterdir():
    if not item.name.startswith('.') and item.name != "ka-ocr.zip":
        print(f"  {item.name}")

Running locally


SSLError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /datasets/irskhirtladze/georgian-synthetic-ocr/resolve/main/ka-ocr.zip (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1010)')))"), '(Request ID: ba08079f-4689-4551-8754-76a8aca74354)')

# Explore data

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(data_dir/"metadata.csv")
print(df.head())
print(df.tail())

                        file_name          text
0  3d_unicode/3d_unicode_0000.png          ამათ
1  3d_unicode/3d_unicode_0001.png   პარტიანკაში
2  3d_unicode/3d_unicode_0002.png  კომენტარების
3  3d_unicode/3d_unicode_0003.png       ფრიდრიხ
4  3d_unicode/3d_unicode_0004.png   ცდწლოოწნწში
                                         file_name      text
100495  NotoSansGeorgian/NotoSansGeorgian_1495.png     რიგში
100496  NotoSansGeorgian/NotoSansGeorgian_1496.png     ალიკა
100497  NotoSansGeorgian/NotoSansGeorgian_1497.png      ტარს
100498  NotoSansGeorgian/NotoSansGeorgian_1498.png     კარგი
100499  NotoSansGeorgian/NotoSansGeorgian_1499.png  სასახლეს


In [4]:
print(df["text"].value_counts())

text
და            4221
არ            1163
რომ            995
იყო            768
კი             604
              ... 
დააპროექტა       1
ცხრილი           1
წულუკიძე         1
ბიჭებთან         1
თცზრმაქ          1
Name: count, Length: 38003, dtype: int64


In [5]:
# Check text length variations
df["text_len"] = df["text"].str.len()
print(df["text_len"].describe())

count    100500.000000
mean          6.390886
std           2.970310
min           2.000000
25%           4.000000
50%           6.000000
75%           8.000000
max          24.000000
Name: text_len, dtype: float64


# Prepare dataset and tokenizer

Checking if trocr already support tokenization for Georgian

In [6]:
from transformers import TrOCRProcessor
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")

# Test Georgian tokenization
test_text = "გამარჯობა"
tokens = processor.tokenizer.tokenize(test_text)
print(tokens)  # If you see lots of <unk> or weird splits, you need a custom tokenizer

2026-01-20 13:11:43.634719: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768914703.882243      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768914703.954849      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768914704.534825      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768914704.534886      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768914704.534890      55 computation_placer.cc:177] computation placer alr

preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

['á', 'ĥ', 'Ĵ', 'á', 'ĥ', 'Ĳ', 'á', 'ĥ', 'Ľ', 'á', 'ĥ', 'Ĳ', 'á', 'ĥ', 'ł', 'á', 'ĥ', '¯', 'á', 'ĥ', 'Ŀ', 'á', 'ĥ', 'ĳ', 'á', 'ĥ', 'Ĳ']


That's not what we need so we'll create custom, character-based tokenizer.

The model predicts next token, in this case token represents char, not a word.

In [7]:
class GeorgianTokenizer:
    def __init__(self, max_length: int = 32):
        # Special tokens
        self.pad_token = "<pad>"
        self.bos_token = "<s>"      # beginning of sequence
        self.eos_token = "</s>"     # end of sequence
        self.unk_token = "<unk>"    # unknown character

        # Georgian alphabet (33 letters)
        self.georgian_chars = "აბგდევზთიკლმნოპჟრსტუფქღყშჩცძწჭხჯჰ"

        # Build vocabulary: special tokens + Georgian characters
        self.vocab = [self.pad_token, self.bos_token, self.eos_token, self.unk_token]
        self.vocab.extend(list(self.georgian_chars))

        # Create mappings
        self.char_to_id = {char: idx for idx, char in enumerate(self.vocab)}
        self.id_to_char = {idx: char for idx, char in enumerate(self.vocab)}

        # Token IDs for special tokens
        self.pad_token_id = 0
        self.bos_token_id = 1
        self.eos_token_id = 2
        self.unk_token_id = 3

        self.max_length = max_length

    def encode(self, text: str, padding: bool = True) -> list[int]:
        """Convert Georgian text to token IDs."""
        # Start with BOS token
        ids = [self.bos_token_id]

        # Convert each character
        for char in text:
            ids.append(self.char_to_id.get(char, self.unk_token_id))

        # Add EOS token
        ids.append(self.eos_token_id)

        # Truncate if too long
        if len(ids) > self.max_length:
            ids = ids[:self.max_length - 1] + [self.eos_token_id]

        # Pad if needed
        if padding:
            ids.extend([self.pad_token_id] * (self.max_length - len(ids)))

        return ids

    def decode(self, ids: list[int]) -> str:
        """Convert token IDs back to text."""
        chars = []
        for id in ids:
            if id in (self.pad_token_id, self.bos_token_id, self.eos_token_id):
                continue
            chars.append(self.id_to_char.get(id, ""))
        return "".join(chars)

    def __len__(self):
        return len(self.vocab)

Test tokenization again

In [8]:
tokenizer = GeorgianTokenizer(max_length=32)

# Test encoding
text = "გამარჯობა"
ids = tokenizer.encode(text)
print(f"Text: {text}")
print(f"IDs: {ids[:15]}...")  # First 15 tokens
print(f"Length: {len(ids)}")

# Test decoding
decoded = tokenizer.decode(ids)
print(f"Decoded: {decoded}")

# Verify vocab size
print(f"Vocab size: {len(tokenizer)}")  # Should be 37 (4 special + 33 Georgian)

Text: გამარჯობა
IDs: [1, 6, 4, 15, 4, 20, 35, 17, 5, 4, 2, 0, 0, 0, 0]...
Length: 32
Decoded: გამარჯობა
Vocab size: 37


Works as expected.

Now we prepare dataset using this tokenizer

In [9]:
import torch
from torch.utils.data import Dataset
from PIL import Image, ImageOps


class GeorgianOCRDataset(Dataset):
    def __init__(self, df: pd.DataFrame, root_dir: str, processor, tokenizer: GeorgianTokenizer):
        self.df = df.reset_index(drop=True)
        self.root_dir = root_dir
        self.processor = processor
        self.tokenizer = tokenizer  # custom tokenizer

    def __len__(self) -> int:
        return len(self.df)

    def __getitem__(self, idx: int) -> dict[str, torch.Tensor]:
        text = self.df.iloc[idx]['text']
        file_path = f"{self.root_dir}/{self.df.iloc[idx]['file_name']}"

        # Open and process image
        img = Image.open(file_path).convert("RGB")
        w, h = img.size
        target_size = 384

        # Scale height to target_size, width proportionally
        scale = target_size / max(w, h)
        new_w, new_h = int(w * scale), int(h * scale)
        img = img.resize((new_w, new_h), Image.Resampling.BILINEAR)

        # Pad to square
        new_img = Image.new("RGB", (target_size, target_size), (255, 255, 255))
        offset = ((target_size - new_w) // 2, (target_size - new_h) // 2)
        new_img.paste(img, offset)

        # Use Processor for Normalization
        pixel_values = self.processor(new_img, return_tensors="pt").pixel_values

        # Tokenize Georgian Text
        labels = self.tokenizer.encode(text)

        # Replace padding token id with -100 so it's ignored by the loss function
        labels = [label if label != self.tokenizer.pad_token_id else -100 for label in labels]

        return {
            "pixel_values": pixel_values.squeeze(),
            "labels": torch.tensor(labels)
        }


Prepare model...

we give it our 37 tokens so model predicts only 37 possible outputs instead of original 50k.

In [10]:
from transformers import VisionEncoderDecoderModel, TrOCRProcessor

# Load processor (for image processing only)
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")

# Create your tokenizer
tokenizer = GeorgianTokenizer(max_length=32)

# Load model and resize token embeddings
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")
model.decoder.resize_token_embeddings(len(tokenizer))  # Resize to 37

# Configure special tokens
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Train test split

In [16]:
from sklearn.model_selection import train_test_split


train_df, test_df = train_test_split(
    df, 
    test_size=0.10, 
    random_state=42, 
    shuffle=True
)

print(train_df["text"].value_counts())

text
და          3843
არ          1043
რომ          896
იყო          696
კი           540
            ... 
ფურცლებს       1
კლერი          1
მოიშორა        1
ნნუხ           1
კარვები        1
Name: count, Length: 35261, dtype: int64
